# Best Trades Backtest Analysis

Analyze historical Best Trades recommendations to:
- Measure win rate and average returns
- Identify which quality ratings/flags have the strongest edge
- Test Vol R:R prediction accuracy
- Optimize entry/exit strategies

## Setup

In [ ]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import sys
import importlib

# Determine root directory
if Path.cwd().name == 'notebooks':
    ROOT = Path.cwd().parent
else:
    ROOT = Path.cwd()

src_path = ROOT / 'src'

# Add src to path
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import backtester
if 'backtest_best_trades' in sys.modules:
    import backtest_best_trades
    importlib.reload(backtest_best_trades)
    print("[OK] Reloaded backtest_best_trades module")

from backtest_best_trades import BestTradesBacktest

print(f"[OK] Root directory: {ROOT}")
print(f"[OK] Backtester loaded")

## Configuration

Set your backtest parameters:
- **category**: Which reports to analyze (sp500, nasdaq100, portfolio, or None for all)
- **holding_period**: Days to hold for performance measurement (7, 14, 30, 60, 90)
- **max_reports**: Maximum number of historical reports to analyze

In [ ]:
# Backtest parameters
CATEGORY = 'sp500'  # sp500, nasdaq100, portfolio, or None for all
HOLDING_PERIOD = 30  # Days to hold
MAX_REPORTS = 10  # Number of historical reports to analyze

print(f"Category: {CATEGORY or 'ALL'}")
print(f"Holding Period: {HOLDING_PERIOD} days")
print(f"Max Reports: {MAX_REPORTS}")

## Initialize Backtester

In [ ]:
# Initialize
results_dir = ROOT / 'scanner_results'
backtester = BestTradesBacktest(results_dir)

# Find available reports
files = backtester.find_best_trades_files(CATEGORY)
print(f"\nFound {len(files)} best trades reports")

if files:
    print("\nMost recent reports:")
    for i, f in enumerate(files[:5], 1):
        date = backtester.extract_date_from_filename(f)
        print(f"  {i}. {f.name} ({date.strftime('%Y-%m-%d')})")

## Run Backtest

This will:
1. Parse each historical best trades report
2. Track actual price performance for each recommendation
3. Calculate returns, max gains, max drawdowns, stop hits, target hits

In [ ]:
# Run backtest
results = backtester.run_backtest(
    category=CATEGORY,
    max_reports=MAX_REPORTS,
    holding_period=HOLDING_PERIOD
)

print(f"\n[OK] Backtest complete!")
print(f"Total trades analyzed: {len(results)}")

## View Sample Results

In [ ]:
# Display sample results
if not results.empty:
    display_cols = ['ticker', 'rank', 'quality', 'quality_flag', 'vol_rr', 
                   f'return_{HOLDING_PERIOD}d', f'mfe_{HOLDING_PERIOD}d', 
                   f'mae_{HOLDING_PERIOD}d', f'stop_hit_{HOLDING_PERIOD}d']
    
    available_cols = [col for col in display_cols if col in results.columns]
    print("\nSample Results (first 10 trades):")
    display(results[available_cols].head(10))

## Edge Analysis

Analyze results to identify the edge:
- Overall performance
- Performance by Quality Rating
- Performance by Quality Flag
- Performance by Vol R:R tier
- Performance by Rank position

In [ ]:
# Generate edge analysis
analysis = backtester.generate_edge_analysis(results, HOLDING_PERIOD)

# Print formatted report
backtester.print_edge_report(analysis)

## Visualizations

### Win Rate by Quality Rating

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Win rate by quality
quality_stats = []
for quality, stats in analysis.get('by_quality', {}).items():
    if stats:
        quality_stats.append({
            'Quality': quality,
            'Win Rate': stats['win_rate'],
            'Avg Return': stats['avg_return'],
            'Count': stats['count']
        })

if quality_stats:
    df_quality = pd.DataFrame(quality_stats)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Win rate
    ax1.bar(df_quality['Quality'], df_quality['Win Rate'], color=['#27ae60', '#3498db'])
    ax1.set_ylabel('Win Rate (%)')
    ax1.set_title('Win Rate by Quality Rating')
    ax1.axhline(y=50, color='r', linestyle='--', alpha=0.5, label='50% baseline')
    ax1.legend()
    
    # Avg return
    colors = ['#27ae60' if x > 0 else '#e74c3c' for x in df_quality['Avg Return']]
    ax2.bar(df_quality['Quality'], df_quality['Avg Return'], color=colors)
    ax2.set_ylabel('Average Return (%)')
    ax2.set_title('Average Return by Quality Rating')
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### Performance by Vol R:R Tier

In [ ]:
# Win rate by Vol R:R tier
rr_stats = []
for tier, stats in analysis.get('by_vol_rr_tier', {}).items():
    if stats:
        rr_stats.append({
            'Vol R:R': tier,
            'Win Rate': stats['win_rate'],
            'Avg Return': stats['avg_return'],
            'Count': stats['count']
        })

if rr_stats:
    df_rr = pd.DataFrame(rr_stats)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Win rate
    colors = ['#e74c3c', '#f39c12', '#27ae60']
    ax1.bar(df_rr['Vol R:R'], df_rr['Win Rate'], color=colors)
    ax1.set_ylabel('Win Rate (%)')
    ax1.set_title('Win Rate by Vol R:R Tier')
    ax1.axhline(y=50, color='r', linestyle='--', alpha=0.5, label='50% baseline')
    ax1.legend()
    
    # Avg return
    bar_colors = ['#27ae60' if x > 0 else '#e74c3c' for x in df_rr['Avg Return']]
    ax2.bar(df_rr['Vol R:R'], df_rr['Avg Return'], color=bar_colors)
    ax2.set_ylabel('Average Return (%)')
    ax2.set_title('Average Return by Vol R:R Tier')
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### Performance by Rank Position

In [ ]:
# Win rate by rank tier
rank_stats = []
for tier, stats in analysis.get('by_rank_tier', {}).items():
    if stats:
        rank_stats.append({
            'Rank': tier,
            'Win Rate': stats['win_rate'],
            'Avg Return': stats['avg_return'],
            'Count': stats['count']
        })

if rank_stats:
    df_rank = pd.DataFrame(rank_stats)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Win rate
    ax1.bar(df_rank['Rank'], df_rank['Win Rate'], color='#3498db')
    ax1.set_ylabel('Win Rate (%)')
    ax1.set_title('Win Rate by Rank Position')
    ax1.axhline(y=50, color='r', linestyle='--', alpha=0.5, label='50% baseline')
    ax1.legend()
    
    # Avg return
    bar_colors = ['#27ae60' if x > 0 else '#e74c3c' for x in df_rank['Avg Return']]
    ax2.bar(df_rank['Rank'], df_rank['Avg Return'], color=bar_colors)
    ax2.set_ylabel('Average Return (%)')
    ax2.set_title('Average Return by Rank Position')
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### Return Distribution

In [ ]:
# Return distribution
return_col = f'return_{HOLDING_PERIOD}d'
if return_col in results.columns:
    returns = results[return_col].dropna()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1.hist(returns, bins=30, color='#3498db', alpha=0.7, edgecolor='black')
    ax1.axvline(x=0, color='r', linestyle='--', label='Break-even')
    ax1.axvline(x=returns.mean(), color='g', linestyle='--', label=f'Mean: {returns.mean():.1f}%')
    ax1.set_xlabel(f'{HOLDING_PERIOD}-Day Return (%)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Return Distribution')
    ax1.legend()
    
    # Box plot
    ax2.boxplot(returns, vert=True)
    ax2.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    ax2.set_ylabel(f'{HOLDING_PERIOD}-Day Return (%)')
    ax2.set_title('Return Box Plot')
    ax2.set_xticklabels(['All Trades'])
    
    plt.tight_layout()
    plt.show()

### Stop Hit Analysis

In [ ]:
# Stop hit rate analysis
stop_col = f'stop_hit_{HOLDING_PERIOD}d'
if stop_col in results.columns:
    stop_data = results[[stop_col, return_col]].dropna()
    
    # Calculate stats
    total_trades = len(stop_data)
    stops_hit = stop_data[stop_col].sum()
    stop_rate = stops_hit / total_trades * 100 if total_trades > 0 else 0
    
    # Returns for stopped vs not stopped
    stopped_returns = stop_data[stop_data[stop_col] == True][return_col]
    not_stopped_returns = stop_data[stop_data[stop_col] == False][return_col]
    
    print(f"\nStop Hit Analysis ({HOLDING_PERIOD} days):")
    print(f"  Total Trades: {total_trades}")
    print(f"  Stops Hit: {stops_hit} ({stop_rate:.1f}%)")
    print(f"  Avg Return (Stopped): {stopped_returns.mean():.2f}%")
    print(f"  Avg Return (Not Stopped): {not_stopped_returns.mean():.2f}%")
    
    # Visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    
    data_to_plot = [stopped_returns, not_stopped_returns]
    labels = [f'Stopped ({len(stopped_returns)})', f'Not Stopped ({len(not_stopped_returns)})']
    
    bp = ax.boxplot(data_to_plot, labels=labels, patch_artist=True)
    bp['boxes'][0].set_facecolor('#e74c3c')
    bp['boxes'][1].set_facecolor('#27ae60')
    
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax.set_ylabel(f'{HOLDING_PERIOD}-Day Return (%)')
    ax.set_title('Returns: Stopped Out vs Not Stopped')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Save Results

In [ ]:
# Save detailed results to CSV
output_file = ROOT / 'backtest_results.csv'
results.to_csv(output_file, index=False)
print(f"[OK] Detailed results saved to: {output_file}")

# Save summary analysis
summary_rows = []

# Overall
overall = analysis.get('overall', {})
if overall:
    summary_rows.append({'Category': 'Overall', 'Segment': 'All', **overall})

# By quality
for quality, stats in analysis.get('by_quality', {}).items():
    if stats:
        summary_rows.append({'Category': 'Quality', 'Segment': quality, **stats})

# By vol rr
for tier, stats in analysis.get('by_vol_rr_tier', {}).items():
    if stats:
        summary_rows.append({'Category': 'Vol R:R', 'Segment': tier, **stats})

# By rank
for tier, stats in analysis.get('by_rank_tier', {}).items():
    if stats:
        summary_rows.append({'Category': 'Rank', 'Segment': tier, **stats})

summary_df = pd.DataFrame(summary_rows)
summary_file = ROOT / 'backtest_summary.csv'
summary_df.to_csv(summary_file, index=False)
print(f"[OK] Summary analysis saved to: {summary_file}")

## Key Insights

Based on the backtest results above, identify:

1. **Does the system have an edge?**
   - Look at overall win rate (>50% is positive)
   - Check average return (positive = edge)
   - Examine win/loss ratio (>1.5x is strong)

2. **Which filters work best?**
   - EXCELLENT vs GOOD quality
   - Quality flags (SAFE ENTRY, IDEAL, etc.)
   - Vol R:R tiers (3+:1 should outperform)
   - Rank position (Top 5 should outperform)

3. **How to improve the edge?**
   - Focus on highest-performing filters only
   - Adjust holding period if needed
   - Test different stop strategies
   - Consider position sizing based on quality